# <center> Лабораторная работа 1 (начало) (Data Structures, Indexing, Cleaning Data, Math)

In [7]:
import pandas as pd
import numpy as np

In [8]:
np.random.seed(242)
### Type your code here


### Информация для последующих заданий
- Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные tr_mcc_codes, tr_types, transactions и gender_train из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк. Обратите внимание на разделители внутри каждого из файлов - они могут различаться!

## Описание данных
### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

#### Task 1.
1. В tr_types выберите произвольные 100 строк с помощью метода sample (указав при этом random_seed равный 242)
2. В полученной на предыдущем этапе подвыборке найдите долю наблюдений (стобец tr_description), в которой содержится подстрока 'плата' (в любом регистре). (\*)

Выведите ответ в виде вещественного числа, округлённого до двух знаков после запятой, отделив дробную часть точкой в формате "123.45"

##### Пояснения:
(\*) Строки "ПлатА за аренду", "ПлатАза аренду", "ПЛАТА" удовлетворяют условию, так как будучи переведёнными в нижний регистр содержат подстроку "плата".

In [9]:
# Импорт библиотеки pandas
import pandas as pd

tr_types = pd.read_csv("tr_types.csv", delimiter=";") # считаем все
tr_types2 = tr_types.sample(100) # Выбираем случаные строки
tr_types2 = tr_types2["tr_description"].str.lower() # нужный столбцеприводем к нижнему регистру
tr_types3 = tr_types2.str.find("плата") # фильтруем ищем где есть плата
print(f"{len(tr_types3) / len(tr_types2):.2f}") # ну и считаем само отношение, считаем фильтрованное к всему

1.00


#### Task 2. 
1. Для поля tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Из перечисленных вариантов выберите те, которые попали в топ-5 транзакций по частоте встречаемости.

#### Выберите все верные пункты:
    - 1) Выдача наличных в АТМ Сбербанк России
    - 2) Комиссия за обслуживание ссудного счета
    - 3) Списание по требованию	
    - 4) Оплата услуги. Банкоматы СБ РФ
    - 5) Погашение кредита (в пределах одного филиала)
    - 6) Покупка. POS ТУ СБ РФ

In [10]:
transactions = pd.read_csv("transactions.csv", delimiter = ",") #Считываем
merged_data = pd.merge(transactions, tr_types, on='tr_type')    # обьеденяем по нужному столбцу   
tr_type_counts = merged_data['tr_description'].value_counts()  #Считаем частоту каждого 
top_5_tr_types = tr_type_counts.head(5)      #Выводим первые 5
print(top_5_tr_types)
# 1 4 6

print('1 4 6 -правильные ответы' )

tr_description
Покупка. POS ТУ СБ РФ                                                                    1589734
Выдача наличных в АТМ Сбербанк России                                                    1040713
Оплата услуги. Банкоматы СБ РФ                                                            970994
Покупка. POS ТУ Россия                                                                    928154
Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя)     691896
Name: count, dtype: int64
1 4 6 -правильные ответы


#### Task 3.
1. В датафрейме transactions задайте столбец  customer_id в качестве индекса.
2. Выделите клиента с максимальной суммой транзакции (то есть с максимальным приходом на карту). (\*)
3. Найдите у него наиболее часто встречающийся модуль суммы приходов/расходов. (\**)


#### Выберите все верные пункты:
    - 1) 1122957.89
    - 2) 15721.41
    - 3) 22459.16
    - 4) 13475494.63
    - 5) 107407.78
    - 6) 65019.26

##### Пояснения:
(\*) Если у клиента были транзакции [-10000, 10, 0, -10], то максимумом будет являться значение 10. <br>
(\**) Если у клиента были транзакции [-10000, 10, 0, -10], то наиболее встречающийся модуль суммы транзакций равен 10, и встретился он 2 раза.

In [11]:

transactions.set_index('customer_id', inplace=True)         # задаем столбец 
max_amount_customer = transactions.groupby('customer_id')['amount'].sum().idxmax() # группиурем по id и находим индекс максимального
max_amount = transactions.loc[max_amount_customer] # выбираем все поля у данного человека
most_amount = max_amount['amount'].abs().mode().values[0] # ищем какая сумма встречается чаще всего
print("Клиент с максимальной суммой транзакции (приходом на карту):", max_amount_customer)
print("Наиболее часто встречающийся модуль суммы приходов/расходов:", most_amount)



Клиент с максимальной суммой транзакции (приходом на карту): 95303296
Наиболее часто встречающийся модуль суммы приходов/расходов: 22459.16


#### Task 4.
1. Найдите максимальную разницу между медианами суммы транзакций, посчитанными при заданных ниже условиях по полю amount из таблицы transactions (\*):
    - Медиана суммы транзакций
    - Медиана суммы транзакций по тем строкам, которые ни в одном из своих столбцов не содержат пустые значения
    - Медиана суммы транзакций по строкам, отсортированным по полю amount в порядке возрастания, и из которых удалены дублирующиеся по столбцам [mcc_code, tr_type] строки, причём при удалении соответстующих дублей остаются только последние из дублирующихся строк (keep='last')

Выведите ответ в виде вещественного числа, округлённого до двух знаков после запятой, отделив дробную часть точкой в формате "123.45"

##### Пояснения:
(\*) Для вычисления максимальной разницы между значениями списка можно использовать функцию np.ptp  
(\**) Если в результате получились значения [1,3,5], то макимальная разница между ними 4 == 5-1.

In [12]:
median_all = transactions['amount'].median() # прост осчитаем медиану
median_no_nulls = transactions.dropna()['amount'].median() # медиана где не содержится не в одной строке пустые значения
# отсортировали по amomunt , удалили дублирующие по столбцам ['mcc_code', 'tr_type'], и посчитали медиану
median_sorted_unique = transactions.sort_values('amount').drop_duplicates(['mcc_code', 'tr_type'], keep='last')['amount'].median()   
max_median = np.ptp([median_all, median_no_nulls, median_sorted_unique]) # считаем максимальную разность

print("{:.2f}".format(max_median))


5366.61
